# Capítulo 5 · Inferencia y grados de libertad

**Objetivo:** analizar la **inferencia estadística** del modelo OLS:

- Definir y verificar los **grados de libertad**:  
  $ Df_{Model} = k, \quad Df_{Residuals} = n - k - 1 $ 
- Interpretar la tabla de coeficientes (`coef`, `std err`, `t`, `P>|t|`, IC95%).  
- Identificar variables significativas y discutir su efecto práctico.


## 5.1 Datos y modelo base

Reutilizamos las **variables candidatas** seleccionadas en el Capítulo 3 y la formulación del modelo OLS del Capítulo 4.

> Nota: los nombres de columnas pueden incluir espacios (por ejemplo, `Gr Liv Area`, `Overall Qual`), pero el procedimiento es general y no depende de nombres fijos.


In [33]:
import numpy as np
import pandas as pd
from pathlib import Path
import statsmodels.api as sm

# Localizar dataset (mismas rutas que en capítulos previos)
CANDIDATE_PATHS = [Path('data/ames_housing.csv'), Path('AmesHousing.csv')]
for p in CANDIDATE_PATHS:
    if p.exists():
        DATA_PATH = p
        break
else:
    raise FileNotFoundError('No se encontró data/ames_housing.csv ni AmesHousing.csv')

df = pd.read_csv(DATA_PATH)

target = "SalePrice"

# Seleccionamos sólo variables numéricas y limpiamos infinitos
num_df = df.select_dtypes(include=[np.number]).replace([np.inf, -np.inf], np.nan)

# Construimos un pool de variables por correlación absoluta con SalePrice
corr_abs = num_df.corr(numeric_only=True)[target].dropna().abs().sort_values(ascending=False)

# Empezamos con top 15 (excluyendo SalePrice) y filtramos colinealidad > 0.85
pool = [c for c in corr_abs.index if c != target][:15]

sel = []
for v in pool:
    if not sel:
        sel.append(v)
        continue
    ok = True
    for u in sel:
        r = abs(num_df[[v, u]].dropna().corr().iloc[0, 1])
        if r > 0.85:
            ok = False
            break
    if ok:
        sel.append(v)
    if len(sel) >= 12:
        break

# Data final para el modelo
data = num_df[[target] + sel].dropna()
y = data[target].values
X = sm.add_constant(data[sel].values, has_constant="add")

sel, X.shape, y.shape

(['Overall Qual',
  'Gr Liv Area',
  'Garage Cars',
  'Total Bsmt SF',
  '1st Flr SF',
  'Year Built',
  'Full Bath',
  'Year Remod/Add',
  'Garage Yr Blt',
  'Mas Vnr Area',
  'TotRms AbvGrd',
  'Fireplaces'],
 (2748, 13),
 (2748,))

Mostramos la lista de **variables candidatas** que se usarán en el modelo:

In [34]:
pd.DataFrame({"Variable": sel, "abs(corr SalePrice)": corr_abs[sel].values}).round(3)

,Variable,abs(corr SalePrice)
0,Overall Qual,0.799
1,Gr Liv Area,0.707
2,Garage Cars,0.648
3,Total Bsmt SF,0.632
4,1st Flr SF,0.622
5,Year Built,0.558
6,Full Bath,0.546
7,Year Remod/Add,0.533
8,Garage Yr Blt,0.527
9,Mas Vnr Area,0.508


## 5.2 Ajuste del modelo OLS y grados de libertad

Ajustamos el modelo mediante `statsmodels.OLS` y verificamos los grados de libertad teóricos:

- $k$: número de predictores (sin contar el intercepto).  
- $n$: número de observaciones.  
- $Df_{Model} = k$.  
- $Df_{Residuals} = n - k - 1$.  


In [35]:
ols = sm.OLS(y, X).fit()

n = X.shape[0]
p = X.shape[1] - 1  # sin contar el intercepto

df_model_teo = p
df_resid_teo = n - p - 1

df_model_teo, df_resid_teo, ols.df_model, ols.df_resid

(12, 2735, 12.0, np.float64(2735.0))

> Observa que los grados de libertad teóricos coinciden con los reportados por `statsmodels`.

## 5.3 Tabla de coeficientes: errores estándar y p-values

Construimos explícitamente la tabla de coeficientes con:

- `coef`: estimador $\hat{\beta}_j$.  
- `std err`: error estándar de $\hat{\beta}_j$.  
- `t`: estadístico t de contraste $H_0: \beta_j = 0$.  
- `P>|t|`: p-valor asociado.  
- `IC 2.5%`, `IC 97.5%`: intervalo de confianza al 95%.  


In [36]:
# Intervalos de confianza (array numpy de shape (k, 2))
ci = ols.conf_int(alpha=0.05)  # ndarray

# Columnas: [lower, upper]
ci_lower = ci[:, 0]
ci_upper = ci[:, 1]

# Nombres de parámetros (constante + variables)
if hasattr(ols.model, "exog_names") and ols.model.exog_names is not None:
    index = ols.model.exog_names
else:
    index = [f"X{i}" for i in range(len(ols.params))]

# Construir la tabla de coeficientes
coef_table = pd.DataFrame(
    {
        "coef":     ols.params,   # ndarray
        "std err":  ols.bse,      # ndarray
        "t":        ols.tvalues,  # ndarray
        "P>|t|":    ols.pvalues,  # ndarray
        "IC 2.5%":  ci_lower,
        "IC 97.5%": ci_upper,
    },
    index=index,
)

coef_table.index.name = "Parámetro"
coef_table.round(4)

,coef,std err,t,P>|t|,IC 2.5%,IC 97.5%
Parámetro,,,,,,
const,-1.257775e+06,94166.7931,-13.3569,0.0000,-1.442420e+06,-1.073130e+06
x1,1.875345e+04,812.2495,23.0883,0.0000,1.716076e+04,2.034613e+04
x2,4.918310e+01,3.0146,16.3147,0.0000,4.327190e+01,5.509430e+01
x3,1.384683e+04,1516.9382,9.1281,0.0000,1.087237e+04,1.682129e+04
x4,2.221390e+01,2.8121,7.8994,0.0000,1.669990e+01,2.772800e+01
x5,1.211340e+01,3.2118,3.7716,0.0002,5.815700e+00,1.841120e+01
x6,2.164772e+02,46.7429,4.6312,0.0000,1.248222e+02,3.081323e+02
x7,-7.950185e+03,1836.4550,-4.3291,0.0000,-1.155116e+04,-4.349206e+03
x8,3.763075e+02,47.1037,7.9889,0.0000,2.839451e+02,4.686698e+02


También podemos ordenar los coeficientes por significancia (p-valor):

In [37]:
coef_sorted = coef_table.sort_values("P>|t|")
coef_sorted.round(4)

,coef,std err,t,P>|t|,IC 2.5%,IC 97.5%
Parámetro,,,,,,
x1,1.875345e+04,812.2495,23.0883,0.0000,1.716076e+04,2.034613e+04
x2,4.918310e+01,3.0146,16.3147,0.0000,4.327190e+01,5.509430e+01
const,-1.257775e+06,94166.7931,-13.3569,0.0000,-1.442420e+06,-1.073130e+06
x3,1.384683e+04,1516.9382,9.1281,0.0000,1.087237e+04,1.682129e+04
x10,3.579160e+01,4.3692,8.1918,0.0000,2.722430e+01,4.435890e+01
x8,3.763075e+02,47.1037,7.9889,0.0000,2.839451e+02,4.686698e+02
x4,2.221390e+01,2.8121,7.8994,0.0000,1.669990e+01,2.772800e+01
x12,8.697666e+03,1236.9640,7.0315,0.0000,6.272187e+03,1.112314e+04
x6,2.164772e+02,46.7429,4.6312,0.0000,1.248222e+02,3.081323e+02


## 5.4 Interpretación de los resultados

Pautas para interpretar la tabla:

1. **Significancia estadística**  
   - Una variable es típicamente considerada significativa si `P>|t| < 0.05`.  
   - Las columnas con intervalos de confianza que **no incluyen 0** refuerzan la evidencia contra $H_0: \beta_j = 0$.  

2. **Magnitud del efecto**  
   - El signo de `coef` indica si el efecto es **directo** (positivo) o **inverso** (negativo) sobre `SalePrice`.  
   - La escala depende de la unidad de la variable (por ejemplo, metros cuadrados, años, número de baños, etc.).  

3. **Intercepto**  
   - Representa el valor esperado del precio cuando todos los predictores valen 0.  
   - En la práctica puede no ser interpretable literalmente, pero se mantiene para centrar el modelo.  


## 5.5 Resumen estadístico completo del modelo

Mostramos el resumen estándar generado por `statsmodels` (incluye R², R² ajustado, pruebas globales, etc.):

In [38]:
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     909.9
Date:                Tue, 11 Nov 2025   Prob (F-statistic):               0.00
Time:                        01:11:48   Log-Likelihood:                -32698.
No. Observations:                2748   AIC:                         6.542e+04
Df Residuals:                    2735   BIC:                         6.550e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.258e+06   9.42e+04    -13.357      0.0

## 5.6 Key takeaways

- Los **grados de libertad** del modelo dependen del número de predictores:  
  $Df_{Model} = k$, $Df_{Residuals} = n - k - 1$.  
- Los errores estándar, estadísticos t y p-valores permiten evaluar la **significancia estadística** de cada coeficiente.  
- Las variables con p-valores pequeños y efectos coherentes con la teoría son buenas candidatas para permanecer en el modelo.  
- Este análisis prepara el terreno para el **diagnóstico de supuestos** (Capítulo 6) y para los **métodos robustos / regularización** en capítulos posteriores.
